In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
# pd.set_option('display.max_rows', 50)
import tabula
from pandasql import sqldf
import math
from scipy.stats import chi2
from scipy import stats
from matplotlib import pyplot as plt

In [2]:
# dna_final = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\dna_layer_final.xlsx')
# dna_final = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\dna_layer_final_new.xlsx')
dna_final = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\dna_layer_final_new_v1.xlsx')
# rna_final = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\rna_layer_final.xlsx')
rna_final_exclude = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\rna_layer_final_exclude.xlsx')
prot_final = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\protein_layer_final.xlsx')
func_final = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\function_layer_final.xlsx')
epig_final = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\epigenomics_fv.xlsx')

----------RNA as val---------------

In [3]:
rna_final_exclude

,gene,score_individual,sig_flag_individual,nonsig_flag_individual,score_kim2020,sig_flag_kim2020,nonsig_flag_kim2020,score_rodrigues2015,sig_flag_rodrigues2015,nonsig_flag_rodrigues2015,score_tripathi2016,sig_flag_tripathi2016,nonsig_flag_tripathi2016,score_wang2021,sig_flag_wang2021,nonsig_flag_wang2021,score_zhou2020,sig_flag_zhou2020,nonsig_flag_zhou2020,score_chen2016,sig_flag_chen2016,nonsig_flag_chen2016,score_leisner2017,sig_flag_leisner2017,nonsig_flag_leisner2017,score_shi2018,sig_flag_shi2018,nonsig_flag_shi2018,score_le2012,sig_flag_le2012,nonsig_flag_le2012,score_total,sigflag_total,nonsigflag_total,score
0,Glyma.06g154400,4.266194,6.0,0.0,4.561853,1.0,0.0,3.568934,1.0,0.0,0.822809,1.0,0.0,1.836406,1.0,0.0,NaN,NaN,NaN,1.481282,1.0,0.0,0.008873,1.0,0.0,2.615471,1.0,0.0,4.099220,2.0,0.0,23.261043,15,0,5.302350
1,Glyma.06g248900,3.429289,10.0,0.0,3.545640,1.0,0.0,0.697926,1.0,0.0,0.972343,1.0,0.0,6.000000,1.0,0.0,0.156669,1.0,0.0,2.368852,1.0,0.0,0.007053,0.0,1.0,4.077786,1.0,0.0,2.002691,2.0,0.0,23.258250,19,1,5.302091
2,Glyma.12g149100,3.999288,9.0,0.0,4.386931,1.0,0.0,0.603691,1.0,0.0,3.571205,1.0,0.0,0.759918,1.0,0.0,NaN,NaN,NaN,1.933510,1.0,0.0,0.003723,0.0,1.0,3.483614,1.0,0.0,1.743978,2.0,0.0,20.485857,17,1,5.045047
3,Glyma.12g221500,3.553189,11.0,0.0,3.838565,1.0,0.0,0.850625,1.0,0.0,2.926320,1.0,0.0,2.459262,1.0,0.0,0.148924,1.0,0.0,3.781446,1.0,0.0,0.005665,0.0,1.0,3.790016,1.0,0.0,4.916125,2.0,0.0,26.270136,20,1,5.581341
4,Glyma.13g279900,4.302588,8.0,0.0,3.771233,1.0,0.0,1.284644,1.0,0.0,1.233600,1.0,0.0,6.000000,1.0,0.0,NaN,NaN,NaN,3.494614,1.0,0.0,0.005758,0.0,1.0,3.970704,1.0,0.0,4.349258,2.0,0.0,28.412401,16,1,5.779963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46708,Glyma.09g124300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001567,0.0,2.0,0.001567,0,2,0.000784
46709,Glyma.06g278900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.314676,0.0,2.0,0.314676,0,2,0.157338
46710,Glyma.06g278300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.428495,1.0,1.0,0.428495,1,1,0.214248
46711,Glyma.10g073000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.028481,0.0,2.0,0.028481,0,2,0.014241


In [4]:
dna_final_v1 = dna_final[['gene', 'sigflag_total', 'nonsigflag_total', 'score']]
rna_final_exclude_v1 = rna_final_exclude[['gene', 'sigflag_total', 'nonsigflag_total', 'score']]
prot_final_v1 = prot_final[['gene', 'sig_flag', 'nonsig_flag', 'score']]
func_final_v1 = func_final[['gene', 'sig_flag', 'nonsig_flag', 'score']]
epig_final_v1 = epig_final[['gene', 'sig_flag', 'nonsig_flag', 'score']]

dna_final_v1 = dna_final_v1.rename(columns = {'sigflag_total':'sigflag_dna', 'nonsigflag_total':'nonsigflag_dna', 'score':'score_dna'})
# dna_final_v1 = dna_final_v1[dna_final_v1['score_dna'].isna() == False].reset_index().drop(['index'], axis = 1)
rna_final_exclude_v1 = rna_final_exclude_v1.rename(columns = {'sigflag_total':'sigflag_rna', 'nonsigflag_total':'nonsigflag_rna', 'score':'score_rna'})
prot_final_v1 = prot_final_v1.rename(columns = {'sig_flag':'sigflag_protein', 'nonsig_flag':'nonsigflag_protein', 'score':'score_protein'})
func_final_v1 = func_final_v1.rename(columns = {'sig_flag':'sigflag_function', 'nonsig_flag':'nonsigflag_function', 'score':'score_function'})
epig_final_v1 = epig_final_v1.rename(columns = {'sig_flag':'sigflag_epigen', 'nonsig_flag':'nonsigflag_epigen', 'score':'score_epigen'})

In [5]:
dna_final_v1['sig_fv_dna'] = dna_final_v1['sigflag_dna'] - dna_final_v1['nonsigflag_dna']
rna_final_exclude_v1['sig_fv_rna'] = rna_final_exclude_v1['sigflag_rna'] - rna_final_exclude_v1['nonsigflag_rna']
prot_final_v1['sig_fv_protein'] = prot_final_v1['sigflag_protein'] - prot_final_v1['nonsigflag_protein']
func_final_v1['sig_fv_function'] = func_final_v1['sigflag_function'] - func_final_v1['nonsigflag_function']
epig_final_v1['sig_fv_epigen'] = epig_final_v1['sigflag_epigen'] - epig_final_v1['nonsigflag_epigen']

dna_final_v1 = dna_final_v1.drop(['sigflag_dna', 'nonsigflag_dna'], axis = 1)
rna_final_exclude_v1 = rna_final_exclude_v1.drop(['sigflag_rna', 'nonsigflag_rna'], axis = 1)
prot_final_v1 = prot_final_v1.drop(['sigflag_protein', 'nonsigflag_protein'], axis = 1)
func_final_v1 = func_final_v1.drop(['sigflag_function', 'nonsigflag_function'], axis = 1)
epig_final_v1 = epig_final_v1.drop(['sigflag_epigen', 'nonsigflag_epigen'], axis = 1)

dna_final_v1.loc[(dna_final_v1['sig_fv_dna'] > 0), 'sig_fv_dna'] = 0.2
dna_final_v1.loc[(dna_final_v1['sig_fv_dna'] < 0), 'sig_fv_dna'] = -0.1
dna_final_v1.loc[(dna_final_v1['sig_fv_dna'] == 0), 'sig_fv_dna'] = 0

rna_final_exclude_v1.loc[(rna_final_exclude_v1['sig_fv_rna'] > 0), 'sig_fv_rna'] = 0.2
rna_final_exclude_v1.loc[(rna_final_exclude_v1['sig_fv_rna'] < 0), 'sig_fv_rna'] = -0.1
rna_final_exclude_v1.loc[(rna_final_exclude_v1['sig_fv_rna'] == 0), 'sig_fv_rna'] = 0

prot_final_v1.loc[(prot_final_v1['sig_fv_protein'] > 0), 'sig_fv_protein'] = 0.2
prot_final_v1.loc[(prot_final_v1['sig_fv_protein'] < 0), 'sig_fv_protein'] = -0.1
prot_final_v1.loc[(prot_final_v1['sig_fv_protein'] == 0), 'sig_fv_protein'] = 0

func_final_v1.loc[(func_final_v1['sig_fv_function'] > 0), 'sig_fv_function'] = 0.2
func_final_v1.loc[(func_final_v1['sig_fv_function'] < 0), 'sig_fv_function'] = -0.1
func_final_v1.loc[(func_final_v1['sig_fv_function'] == 0), 'sig_fv_function'] = 0

epig_final_v1.loc[(epig_final_v1['sig_fv_epigen'] > 0), 'sig_fv_epigen'] = 0.2
epig_final_v1.loc[(epig_final_v1['sig_fv_epigen'] < 0), 'sig_fv_epigen'] = -0.1
epig_final_v1.loc[(epig_final_v1['sig_fv_epigen'] == 0), 'sig_fv_epigen'] = 0

In [6]:
data_final = dna_final_v1.merge(rna_final_exclude_v1, how = 'outer', on = ['gene']).merge(prot_final_v1, how = 'outer', on = ['gene']).merge(func_final_v1, how = 'outer', on = ['gene']).merge(epig_final_v1, how = 'outer', on = ['gene'])[['gene', 'score_dna', 'score_rna', 'score_protein', 'score_function', 'score_epigen', 'sig_fv_dna', 'sig_fv_rna', 'sig_fv_protein', 'sig_fv_function', 'sig_fv_epigen']]

In [7]:
data_final['score'] = data_final[['score_dna', 'score_rna', 'score_protein', 'score_function', 'score_epigen']].sum(axis = 1)
data_final['weight'] = data_final[['sig_fv_dna', 'sig_fv_rna', 'sig_fv_protein', 'sig_fv_function', 'sig_fv_epigen']].sum(axis = 1)
data_final['weight'] = 1 + data_final['weight']

In [8]:
data_final['dna_flag'] = ''
data_final['rna_flag'] = ''
data_final['protein_flag'] = ''
data_final['function_flag'] = ''
data_final['epigen_flag'] = ''
data_final.loc[(data_final['score_dna'].isna() == False), 'dna_flag'] = 1
data_final.loc[(data_final['score_dna'].isna()), 'dna_flag'] = 0
data_final.loc[(data_final['score_rna'].isna() == False), 'rna_flag'] = 1
data_final.loc[(data_final['score_rna'].isna()), 'rna_flag'] = 0
data_final.loc[(data_final['score_protein'].isna() == False), 'protein_flag'] = 1
data_final.loc[(data_final['score_protein'].isna()), 'protein_flag'] = 0
data_final.loc[(data_final['score_function'].isna() == False), 'function_flag'] = 1
data_final.loc[(data_final['score_function'].isna()), 'function_flag'] = 0
data_final.loc[(data_final['score_epigen'].isna() == False), 'epigen_flag'] = 1
data_final.loc[(data_final['score_epigen'].isna()), 'epigen_flag'] = 0

data_final['total_layers'] = data_final['dna_flag'] + data_final['rna_flag'] + data_final['protein_flag'] + data_final['function_flag'] + data_final['epigen_flag']

In [9]:
data_final['weight_score'] = data_final['score'] * data_final['weight']
data_final = data_final.rename(columns = {'score':'unweight_score'})

In [10]:
dataf_w = data_final.sort_values(['weight_score'], ascending = False).reset_index().drop(['index'], axis = 1)
dataf_uw = data_final.sort_values(['unweight_score'], ascending = False).reset_index().drop(['index'], axis = 1)

In [11]:
dataf_w = dataf_w.drop_duplicates(['gene']).reset_index().drop(['index'], axis = 1)
dataf_uw = dataf_uw.drop_duplicates(['gene']).reset_index().drop(['index'], axis = 1)

In [12]:
# writer = pd.ExcelWriter(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\integrated_results_DNAinside.xlsx')
# dataf_w.to_excel(writer, sheet_name = 'weighted', index = False)
# dataf_uw.to_excel(writer, sheet_name = 'unweighted', index = False)
# writer.close()

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\edwar\\Desktop\\農藝\\paper2\\data collection\\integrated_results_DNAinside.xlsx'

In [13]:
# val data
shin_2015 = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\whole genome data\complete\gene expression\Shin-2015\Shin2015_wgd.xlsx')
tamang_2021 = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\whole genome data\complete\gene expression\Tamang-2021\Tamang2021_wgd.xlsx')

In [14]:
from scipy import stats

-----shin2015------

In [15]:
glyma_gene_ver = pd.read_csv(r'C:\Users\edwar\Desktop\農藝\paper2\Glyma_genes_correspondence_final.txt', delimiter = '\t')

In [16]:
shin_2015['drought_treatment_span'].unique()

array([ 6, 12, 24], dtype=int64)

In [17]:
shin_2015_6h = shin_2015[shin_2015['drought_treatment_span'] == 6].reset_index().drop(['index'], axis = 1)[['mapped_gene', 'fc']].rename(columns = {'mapped_gene':'gene_org'})
shin_2015_12h = shin_2015[shin_2015['drought_treatment_span'] == 12].reset_index().drop(['index'], axis = 1)[['mapped_gene', 'fc']].rename(columns = {'mapped_gene':'gene_org'})
shin_2015_24h = shin_2015[shin_2015['drought_treatment_span'] == 24].reset_index().drop(['index'], axis = 1)[['mapped_gene', 'fc']].rename(columns = {'mapped_gene':'gene_org'})

shin_2015_6h = shin_2015_6h.merge(glyma_gene_ver, left_on = ['gene_org'], right_on = ['Wm82.a1.v1'], how = 'left').drop(['Wm82.a1.v1', 'Wm82.a1.v1.1'], axis = 1).rename(columns = {'Wm82.a2.v1':'gene'})
shin_2015_12h = shin_2015_12h.merge(glyma_gene_ver, left_on = ['gene_org'], right_on = ['Wm82.a1.v1'], how = 'left').drop(['Wm82.a1.v1', 'Wm82.a1.v1.1'], axis = 1).rename(columns = {'Wm82.a2.v1':'gene'})
shin_2015_24h = shin_2015_24h.merge(glyma_gene_ver, left_on = ['gene_org'], right_on = ['Wm82.a1.v1'], how = 'left').drop(['Wm82.a1.v1', 'Wm82.a1.v1.1'], axis = 1).rename(columns = {'Wm82.a2.v1':'gene'})

shin_2015_6h = shin_2015_6h[shin_2015_6h['gene'].isna() == False].reset_index().drop(['index'], axis = 1)
shin_2015_12h = shin_2015_12h[shin_2015_12h['gene'].isna() == False].reset_index().drop(['index'], axis = 1)
shin_2015_24h = shin_2015_24h[shin_2015_24h['gene'].isna() == False].reset_index().drop(['index'], axis = 1)

shin_2015_6h.loc[(shin_2015_6h['fc'] < 1), 'fc'] = 1 / shin_2015_6h['fc']
shin_2015_12h.loc[(shin_2015_12h['fc'] < 1), 'fc'] = 1 / shin_2015_12h['fc']
shin_2015_24h.loc[(shin_2015_24h['fc'] < 1), 'fc'] = 1 / shin_2015_24h['fc']

shin_2015_6h['p_val'] = 0.1 ** np.log2(shin_2015_6h['fc'])
shin_2015_12h['p_val'] = 0.1 ** np.log2(shin_2015_12h['fc'])
shin_2015_24h['p_val'] = 0.1 ** np.log2(shin_2015_24h['fc'])

shin_2015_6h['-logP'] = -np.log10(shin_2015_6h['p_val'])
shin_2015_12h['-logP'] = -np.log10(shin_2015_12h['p_val'])
shin_2015_24h['-logP'] = -np.log10(shin_2015_24h['p_val'])

In [21]:
# DNA_inside_w
n = 135
gene_list_uw = list(dataf_uw.iloc[:n]['gene'])
gene_list_w = list(dataf_w.iloc[:n]['gene'])

In [22]:
# validation performed on 6hr drt
p_val = stats.ttest_ind(list(shin_2015_6h[shin_2015_6h['gene'].isin(gene_list_uw)]['-logP']), shin_2015_6h['-logP'], alternative = 'greater')[1]
print('6h drt uw p:' + str(p_val))

p_val = stats.ttest_ind(list(shin_2015_6h[shin_2015_6h['gene'].isin(gene_list_w)]['-logP']), shin_2015_6h['-logP'], alternative = 'greater')[1]
print('6h drt w p:' + str(p_val))

6h drt uw p:6.73310418555902e-33
6h drt w p:1.5215185159821285e-32


In [23]:
# validation performed on 6hr drt
p_val = stats.ranksums(list(shin_2015_6h[shin_2015_6h['gene'].isin(gene_list_uw)]['-logP']), shin_2015_6h['-logP'], alternative = 'greater')[1]
print('6h drt uw p:' + str(p_val))

p_val = stats.ranksums(list(shin_2015_6h[shin_2015_6h['gene'].isin(gene_list_w)]['-logP']), shin_2015_6h['-logP'], alternative = 'greater')[1]
print('6h drt w p:' + str(p_val))

6h drt uw p:3.4710884041990372e-15
6h drt w p:5.970750962459492e-13


In [24]:
p_val = stats.ttest_ind(list(shin_2015_12h[shin_2015_12h['gene'].isin(gene_list_uw)]['-logP']), shin_2015_12h['-logP'], alternative = 'greater')[1]
print('12h drt uw p:' + str(p_val))

p_val = stats.ttest_ind(list(shin_2015_12h[shin_2015_12h['gene'].isin(gene_list_w)]['-logP']), shin_2015_12h['-logP'], alternative = 'greater')[1]
print('12h drt w p:' + str(p_val))

12h drt uw p:5.119990449597535e-45
12h drt w p:4.349805691007628e-50


In [25]:
p_val = stats.ranksums(list(shin_2015_12h[shin_2015_12h['gene'].isin(gene_list_uw)]['-logP']), shin_2015_12h['-logP'], alternative = 'greater')[1]
print('12h drt uw p:' + str(p_val))

p_val = stats.ranksums(list(shin_2015_12h[shin_2015_12h['gene'].isin(gene_list_w)]['-logP']), shin_2015_12h['-logP'], alternative = 'greater')[1]
print('12h drt w p:' + str(p_val))

12h drt uw p:5.973491617815594e-19
12h drt w p:4.4062860405731845e-19


In [26]:
p_val = stats.ttest_ind(list(shin_2015_24h[shin_2015_24h['gene'].isin(gene_list_uw)]['-logP']), shin_2015_24h['-logP'], alternative = 'greater')[1]
print('24h drt uw p:' + str(p_val))

p_val = stats.ttest_ind(list(shin_2015_24h[shin_2015_24h['gene'].isin(gene_list_w)]['-logP']), shin_2015_24h['-logP'], alternative = 'greater')[1]
print('24h drt w p:' + str(p_val))

24h drt uw p:1.60424283170731e-31
24h drt w p:8.465013765270267e-35


In [27]:
p_val = stats.ranksums(list(shin_2015_24h[shin_2015_24h['gene'].isin(gene_list_uw)]['-logP']), shin_2015_24h['-logP'], alternative = 'greater')[1]
print('24h drt uw p:' + str(p_val))

p_val = stats.ranksums(list(shin_2015_24h[shin_2015_24h['gene'].isin(gene_list_w)]['-logP']), shin_2015_24h['-logP'], alternative = 'greater')[1]
print('24h drt w p:' + str(p_val))

24h drt uw p:4.8053846437949565e-15
24h drt w p:1.3828654441357628e-14


-----tamang2021------

In [28]:
tamang_2021_5d = tamang_2021[tamang_2021['drought_treatment_span'] == 120].reset_index().drop(['index'], axis = 1)[['mapped_gene', 'p', 'fc']].reset_index().drop(['index'], axis = 1).rename(columns = {'mapped_gene':'gene'})
tamang_2021_7d = tamang_2021[tamang_2021['drought_treatment_span'] == 144].reset_index().drop(['index'], axis = 1)[['mapped_gene', 'p', 'fc']].reset_index().drop(['index'], axis = 1).rename(columns = {'mapped_gene':'gene'})
tamang_2021_5d = tamang_2021_5d[tamang_2021_5d['fc'].isna() == False].reset_index().drop(['index'], axis = 1)
tamang_2021_7d = tamang_2021_7d[tamang_2021_7d['fc'].isna() == False].reset_index().drop(['index'], axis = 1)

tamang_2021_5d.loc[(tamang_2021_5d['fc'] < 1), 'fc'] = 1 / tamang_2021_5d['fc']
tamang_2021_7d.loc[(tamang_2021_7d['fc'] < 1), 'fc'] = 1 / tamang_2021_7d['fc']

tamang_2021_5d['p_val'] = 0.1 ** np.log2(tamang_2021_5d['fc'])
tamang_2021_7d['p_val'] = 0.1 ** np.log2(tamang_2021_7d['fc'])

tamang_2021_5d['-logP'] = -np.log10(tamang_2021_5d['p_val'])
tamang_2021_7d['-logP'] = -np.log10(tamang_2021_7d['p_val'])

In [30]:
# DNA_inside_w
n = 135
gene_list_uw = list(dataf_uw.iloc[:n]['gene'])
gene_list_w = list(dataf_w.iloc[:n]['gene'])

In [31]:
# tamang_2021_5d[tamang_2021_5d['gene'].isin(gene_list_uw)]['-logP']
p_val = stats.ttest_ind(tamang_2021_5d[tamang_2021_5d['gene'].isin(gene_list_uw)]['-logP'], tamang_2021_5d['-logP'], alternative = 'greater')[1]
print('5d drt uw p:' + str(p_val))

p_val = stats.ttest_ind(tamang_2021_5d[tamang_2021_5d['gene'].isin(gene_list_w)]['-logP'], tamang_2021_5d['-logP'], alternative = 'greater')[1]
print('5d drt ww p:' + str(p_val))

5d drt uw p:1.922702908453669e-53
5d drt ww p:1.1940341487613139e-28


In [32]:
p_val = stats.ranksums(tamang_2021_5d[tamang_2021_5d['gene'].isin(gene_list_uw)]['-logP'], tamang_2021_5d['-logP'], alternative = 'greater')[1]
print('5d drt uw p:' + str(p_val))

p_val = stats.ranksums(tamang_2021_5d[tamang_2021_5d['gene'].isin(gene_list_w)]['-logP'], tamang_2021_5d['-logP'], alternative = 'greater')[1]
print('5d drt ww p:' + str(p_val))

5d drt uw p:1.4881645494207293e-25
5d drt ww p:1.9580553670878118e-17


In [33]:
p_val = stats.ttest_ind(tamang_2021_7d[tamang_2021_7d['gene'].isin(gene_list_uw)]['-logP'], tamang_2021_7d['-logP'], alternative = 'greater')[1]
print('7d drt uw p:' + str(p_val))

p_val = stats.ttest_ind(tamang_2021_7d[tamang_2021_7d['gene'].isin(gene_list_w)]['-logP'], tamang_2021_7d['-logP'], alternative = 'greater')[1]
print('7d drt ww p:' + str(p_val))

7d drt uw p:7.287416190152004e-47
7d drt ww p:4.666328639681657e-27


In [34]:
p_val = stats.ranksums(tamang_2021_7d[tamang_2021_7d['gene'].isin(gene_list_uw)]['-logP'], tamang_2021_7d['-logP'], alternative = 'greater')[1]
print('7d drt uw p:' + str(p_val))

p_val = stats.ranksums(tamang_2021_7d[tamang_2021_7d['gene'].isin(gene_list_w)]['-logP'], tamang_2021_7d['-logP'], alternative = 'greater')[1]
print('7d drt ww p:' + str(p_val))

7d drt uw p:1.065091421007716e-21
7d drt ww p:1.3165129343760308e-14


In [35]:
len(set(gene_list_uw) & set(gene_list_w))

97

In [2]:
data = pd.read_excel(r"C:\Users\edwar\Desktop\農藝\paper2\data collection\integrated_results_DNAinside.xlsx")

In [4]:
dataf_w = data.sort_values(['weight_score'], ascending = False).reset_index().drop(['index'], axis = 1)
dataf_uw = data.sort_values(['unweight_score'], ascending = False).reset_index().drop(['index'], axis = 1)

In [11]:
gene_list_w = list(dataf_w[:135]['gene'])
gene_list_uw = list(dataf_uw[:135]['gene'])
print(len(set(gene_list_w) & set(gene_list_uw)))

97


In [6]:
dataf_w[:135]

,gene,score_dna,score_rna,score_protein,score_function,score_epigen,sig_fv_dna,sig_fv_rna,sig_fv_protein,sig_fv_function,sig_fv_epigen,unweight_score,weight,dna_flag,rna_flag,protein_flag,function_flag,epigen_flag,total_layers,weight_score
0,Glyma.07g100500,3.694529,2.220309,4.980167,5.275634,NaN,0.2,0.2,0.2,0.2,NaN,16.170640,1.8,1,1,1,1,0,4,29.107152
1,Glyma.13g208000,4.097316,5.249983,4.775809,3.300000,NaN,0.2,0.2,0.2,0.0,NaN,17.423108,1.6,1,1,1,1,0,4,27.876972
2,Glyma.04g200900,4.463782,2.172194,3.773508,5.025963,NaN,0.2,0.2,0.2,0.2,NaN,15.435447,1.8,1,1,1,1,0,4,27.783805
3,Glyma.05g128000,2.730402,3.214632,4.444123,4.986369,NaN,0.2,0.2,0.2,0.2,NaN,15.375526,1.8,1,1,1,1,0,4,27.675947
4,Glyma.14g031000,3.198549,3.582737,4.168967,3.436906,NaN,0.2,0.2,0.2,0.2,NaN,14.387159,1.8,1,1,1,1,0,4,25.896886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Glyma.03g171100,3.042172,1.632970,4.063538,3.233926,NaN,0.2,0.2,0.2,0.0,NaN,11.972606,1.6,1,1,1,1,0,4,19.156169
131,Glyma.01g219400,2.377474,4.315255,5.271045,NaN,NaN,0.2,0.2,0.2,NaN,NaN,11.963774,1.6,1,1,1,0,0,3,19.142038
132,Glyma.18g018600,4.404594,4.145560,3.400000,NaN,NaN,0.2,0.2,0.2,NaN,NaN,11.950155,1.6,1,1,1,0,0,3,19.120247
133,Glyma.16g173100,4.325642,1.771685,2.823909,3.028728,NaN,0.2,0.2,0.2,0.0,NaN,11.949963,1.6,1,1,1,1,0,4,19.119941


In [9]:
dataf_uw[:135]

,gene,score_dna,score_rna,score_protein,score_function,score_epigen,sig_fv_dna,sig_fv_rna,sig_fv_protein,sig_fv_function,sig_fv_epigen,unweight_score,weight,dna_flag,rna_flag,protein_flag,function_flag,epigen_flag,total_layers,weight_score
0,Glyma.13g208000,4.097316,5.249983,4.775809,3.300000,NaN,0.2,0.2,0.2,0.0,NaN,17.423108,1.6,1,1,1,1,0,4,27.876972
1,Glyma.07g100500,3.694529,2.220309,4.980167,5.275634,NaN,0.2,0.2,0.2,0.2,NaN,16.170640,1.8,1,1,1,1,0,4,29.107152
2,Glyma.07g033500,5.831174,4.432774,1.160815,4.500000,NaN,0.2,0.2,-0.1,0.0,NaN,15.924763,1.3,1,1,1,1,0,4,20.702192
3,Glyma.17g072400,1.437624,5.317399,5.463652,3.300000,NaN,0.2,0.2,0.2,0.0,NaN,15.518675,1.6,1,1,1,1,0,4,24.829880
4,Glyma.04g200900,4.463782,2.172194,3.773508,5.025963,NaN,0.2,0.2,0.2,0.2,NaN,15.435447,1.8,1,1,1,1,0,4,27.783805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,Glyma.09g173500,4.042109,0.693253,4.422182,3.300000,NaN,0.2,0.0,0.2,0.0,NaN,12.457544,1.4,1,1,1,1,0,4,17.440562
131,Glyma.08g246400,3.300589,4.511020,1.000000,3.644323,NaN,0.2,0.2,0.0,0.0,NaN,12.455932,1.4,1,1,1,1,0,4,17.438305
132,Glyma.13g270600,2.434430,2.864467,3.803788,3.346299,NaN,0.2,0.2,0.2,0.2,NaN,12.448983,1.8,1,1,1,1,0,4,22.408170
133,Glyma.16g212500,2.037846,2.398344,4.689308,3.300000,NaN,0.2,0.2,0.2,0.0,NaN,12.425498,1.6,1,1,1,1,0,4,19.880797
